In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import os.path


class ImageDataset(Dataset):
    def __init__(self, df, load_dir, transform):
        self.df = df
        self.load_dir = load_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        image_path = self.load_dir + "/Data/" + self.df.iloc[index,0]
        image = Image.open(image_path)
        tensor_image = self.transform(image)
        label = self.df.iloc[index,1]
        return tensor_image, label


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5) # input, output, kernal size 
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.conv3 = nn.Conv2d(32, 32, 5)
        self.fc1 = nn.Linear(32*114*114, 128)
        self.fc2 = nn.Linear(128, 84)
        self.fc3 = nn.Linear(84,10)
        self.dp1 = nn.Dropout(0.5)
        self.dp2 = nn.Dropout(0.5)
        self.dp3 = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dp1(x)
        #print(x.shape)
        x = x.view(-1, 32*114*114)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dp3(x)
        x = self.fc3(x)
        # x = F.log_softmax(x, dim=1)
        return x

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
num_workers = 12
num_epoch = 10
batch_size = 128
learning_rate = 0.001


load_dir = "/content/drive/MyDrive/Projects/wikiart"
#load_dir = "/content/drive/.shortcut-targets-by-id/162Kf_zOQOfxR3jWEwQzxFxYLZbKOqb4S/WikiArt"
df_train = pd.read_csv(load_dir+'/Labels/Genre/genre_train.csv')
df_val = pd.read_csv(load_dir+'/Labels/Genre/genre_val.csv')
transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),])
data_train = ImageDataset(df_train, load_dir, transform)
data_val = ImageDataset(df_val, load_dir, transform)
dataloader_train = DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
dataloader_val = DataLoader(data_val, batch_size=batch_size, shuffle=False, num_workers=num_workers)


model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Lists to store accuracy values
train_acc_list = []
val_acc_list = []


for epoch in range(num_epoch):
    batch_accuracies = []
    for batch_idx, (images, labels) in enumerate(tqdm(dataloader_train)):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        correct = (predicted == labels).sum().item()
        accuracy = correct / labels.size(0) * 100
        train_acc_list.append(accuracy)
        batch_accuracies.append(accuracy)
    
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for images, labels in dataloader_val:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()
    val_accuracy = total_correct / total_samples * 100
    val_acc_list.append(val_accuracy)
    for count, accuracy in enumerate(batch_accuracies):
        if count % int(len(dataloader_train)/10) == 0:
            print(f"Epoch [{epoch+1}/{num_epoch}], Batch [{count}/{len(dataloader_train)}], Train Accuracy: {accuracy:.2f}%")
    print(f"Epoch [{epoch+1}/{num_epoch}], Validation Accuracy: {val_accuracy:.2f}%")
    model.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
 17%|█▋        | 62/356 [06:18<04:58,  1.01s/it]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 82%|████████▏ | 291/356 [27:03<17:15, 15.93s/it]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████|

Epoch [1/10], Batch [0/356], Train Accuracy: 10.16%
Epoch [1/10], Batch [35/356], Train Accuracy: 26.56%
Epoch [1/10], Batch [70/356], Train Accuracy: 30.47%
Epoch [1/10], Batch [105/356], Train Accuracy: 35.16%
Epoch [1/10], Batch [140/356], Train Accuracy: 37.50%
Epoch [1/10], Batch [175/356], Train Accuracy: 37.50%
Epoch [1/10], Batch [210/356], Train Accuracy: 42.97%
Epoch [1/10], Batch [245/356], Train Accuracy: 36.72%
Epoch [1/10], Batch [280/356], Train Accuracy: 39.06%
Epoch [1/10], Batch [315/356], Train Accuracy: 34.38%
Epoch [1/10], Batch [350/356], Train Accuracy: 32.81%
Epoch [1/10], Validation Accuracy: 38.04%


 51%|█████     | 181/356 [05:09<01:17,  2.26it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 81%|████████  | 289/356 [08:02<00:28,  2.35it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████| 356/356 [09:34<00:00,  1.61s/it]


Epoch [2/10], Batch [0/356], Train Accuracy: 33.59%
Epoch [2/10], Batch [35/356], Train Accuracy: 35.94%
Epoch [2/10], Batch [70/356], Train Accuracy: 35.94%
Epoch [2/10], Batch [105/356], Train Accuracy: 33.59%
Epoch [2/10], Batch [140/356], Train Accuracy: 47.66%
Epoch [2/10], Batch [175/356], Train Accuracy: 39.84%
Epoch [2/10], Batch [210/356], Train Accuracy: 40.62%
Epoch [2/10], Batch [245/356], Train Accuracy: 38.28%
Epoch [2/10], Batch [280/356], Train Accuracy: 35.94%
Epoch [2/10], Batch [315/356], Train Accuracy: 39.06%
Epoch [2/10], Batch [350/356], Train Accuracy: 43.75%
Epoch [2/10], Validation Accuracy: 41.84%


  2%|▏         | 6/356 [00:22<08:44,  1.50s/it]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 30%|██▉       | 106/356 [03:12<02:20,  1.78it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████| 356/356 [10:01<00:00,  1.69s/it]


...

Epoch [8/10], Batch [0/356], Train Accuracy: 61.72%
Epoch [8/10], Batch [35/356], Train Accuracy: 58.59%
Epoch [8/10], Batch [70/356], Train Accuracy: 57.81%
Epoch [8/10], Batch [105/356], Train Accuracy: 56.25%
Epoch [8/10], Batch [140/356], Train Accuracy: 60.16%
Epoch [8/10], Batch [175/356], Train Accuracy: 66.41%
Epoch [8/10], Batch [210/356], Train Accuracy: 60.94%
Epoch [8/10], Batch [245/356], Train Accuracy: 60.16%
Epoch [8/10], Batch [280/356], Train Accuracy: 58.59%
Epoch [8/10], Batch [315/356], Train Accuracy: 65.62%
Epoch [8/10], Batch [350/356], Train Accuracy: 65.62%
Epoch [8/10], Validation Accuracy: 47.42%


 27%|██▋       | 96/356 [02:54<02:11,  1.98it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 54%|█████▍    | 192/356 [05:35<02:08,  1.27it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████| 356/356 [09:50<00:00,  1.66s/it]


Epoch [9/10], Batch [0/356], Train Accuracy: 63.28%
Epoch [9/10], Batch [35/356], Train Accuracy: 67.19%
Epoch [9/10], Batch [70/356], Train Accuracy: 68.75%
Epoch [9/10], Batch [105/356], Train Accuracy: 66.41%
Epoch [9/10], Batch [140/356], Train Accuracy: 60.16%
Epoch [9/10], Batch [175/356], Train Accuracy: 57.81%
Epoch [9/10], Batch [210/356], Train Accuracy: 60.94%
Epoch [9/10], Batch [245/356], Train Accuracy: 53.91%
Epoch [9/10], Batch [280/356], Train Accuracy: 60.94%
Epoch [9/10], Batch [315/356], Train Accuracy: 60.94%
Epoch [9/10], Batch [350/356], Train Accuracy: 62.50%
Epoch [9/10], Validation Accuracy: 48.58%


 40%|████      | 144/356 [04:13<02:05,  1.69it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 46%|████▌     | 164/356 [04:56<04:14,  1.33s/it]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████| 356/356 [10:05<00:00,  1.70s/it]


Epoch [10/10], Batch [0/356], Train Accuracy: 71.09%
Epoch [10/10], Batch [35/356], Train Accuracy: 73.44%
Epoch [10/10], Batch [70/356], Train Accuracy: 67.19%
Epoch [10/10], Batch [105/356], Train Accuracy: 64.06%
Epoch [10/10], Batch [140/356], Train Accuracy: 64.84%
Epoch [10/10], Batch [175/356], Train Accuracy: 55.47%
Epoch [10/10], Batch [210/356], Train Accuracy: 65.62%
Epoch [10/10], Batch [245/356], Train Accuracy: 68.75%
Epoch [10/10], Batch [280/356], Train Accuracy: 58.59%
Epoch [10/10], Batch [315/356], Train Accuracy: 67.19%
Epoch [10/10], Batch [350/356], Train Accuracy: 64.06%
Epoch [10/10], Validation Accuracy: 48.41%
